In [1]:
# ============================================================================
# CÉLULA 1: Importações e Configurações
# ============================================================================
import d3rlpy
import numpy as np
import pandas as pd
import joblib
import torch
import os
from d3rlpy.algos import CQLConfig
from d3rlpy.models import QRQFunctionFactory
from d3rlpy.dataset import Episode
# Esta é a linha que faltava ou estava errada:
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer, Episode 
from d3rlpy.metrics import AverageValueEstimationEvaluator
from sklearn.model_selection import train_test_split
# 'average_value_estimation_scorer' is replaced by this Evaluator class
from d3rlpy.metrics import AverageValueEstimationEvaluator

# FQE is no longer a scorer. It is an OPE algorithm in 'd3rlpy.ope'.
# We import it here to use it separately later, or remove it if you only wanted a simple metric.
from d3rlpy.ope import FQEConfig

from sklearn.model_selection import train_test_split
import d3rlpy
# Configuração de sementes
d3rlpy.seed(42)
np.random.seed(42)

print(f"d3rlpy version: {d3rlpy.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"GPU disponível? {torch.cuda.is_available()}")
print(f"d3rlpy version: {d3rlpy.__version__}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


d3rlpy version: 2.8.1

Torch version: 2.9.1+cpu

GPU disponível? False

d3rlpy version: 2.8.1

In [2]:

# ============================================================================
# CÉLULA 2: Carregar Artefatos Gerados (Buffer e Scalers)
# ============================================================================
BUFFER_PATH = "rl_offline_buffer.h5"

print(f"\n[1/5] Carregando ReplayBuffer de '{BUFFER_PATH}' ...")

# Carrega o buffer gerado pelo Generator_NEW.py
# Precisamos recriar a estrutura vazia e carregar o arquivo
with open(BUFFER_PATH, "rb") as f:
    # O limit deve ser igual ou maior que o tamanho do dataset original
    dataset = ReplayBuffer.load(f, FIFOBuffer(limit=200000))

print(f"✓ Buffer carregado com sucesso.")
print(f"  # Episódios: {len(dataset.episodes)}")
print(f"  # Transições totais: {dataset.transition_count}")

# Carrega os Scalers para pós-processamento e métricas reais
print("\n[2/5] Carregando Scalers...")
try:
    scaler_estado = joblib.load("scaler_estado.joblib")
    scaler_acao = joblib.load("scaler_acao.joblib")
    scaler_recompensa = joblib.load("scaler_recompensa.joblib")
    # OHE para referência
    ohe = joblib.load("ohe_encoder.joblib")
    print("✓ Scalers carregados.")
except FileNotFoundError:
    print("❌ ERRO: Scalers não encontrados. Execute Generator_NEW.py primeiro.")


[1/5] Carregando ReplayBuffer de 'rl_offline_buffer.h5' ...

2025-11-27 13:57.59

 [

info     

] 

...

✓ Buffer carregado com sucesso.

  # Episódios: 1

  # Transições totais: 99999


[2/5] Carregando Scalers...

✓ Scalers carregados.

In [3]:
# ============================================================================
# CÉLULA 3: Divisão Treino / Validação (Split Robusto - CORREÇÃO FINAL)
# ============================================================================
print("\n[3/5] Dividindo dados em Treino e Teste (Hold-out)...")

# Verifica se temos poucos episódios (ex: 1 episódio gigante)
if len(dataset.episodes) < 10:
    print(f"⚠️ Aviso: Encontrados apenas {len(dataset.episodes)} episódios.")
    print("   -> Dividindo o episódio gigante em sub-episódios menores para validação...")
    
    giant_ep = dataset.episodes[0]
    
    # Acessa arrays
    obs = giant_ep.observations
    act = giant_ep.actions
    rew = giant_ep.rewards
    
    # Reconstrói terminais (assumindo apenas o final como terminal)
    term = np.zeros((len(obs), 1), dtype=np.float32)
    # Tenta pegar atributo terminal ou terminated
    is_global_term = getattr(giant_ep, 'terminal', getattr(giant_ep, 'terminated', True))
    if is_global_term:
        term[-1] = 1.0
        
    total_len = len(obs)
    split_idx = int(total_len * 0.8)
    
    # Divide manualmente
    train_obs, test_obs = obs[:split_idx], obs[split_idx:]
    train_act, test_act = act[:split_idx], act[split_idx:]
    train_rew, test_rew = rew[:split_idx], rew[split_idx:]
    train_term, test_term = term[:split_idx], term[split_idx:]
    
    from d3rlpy.dataset import Episode
    
    # Helper seguro (POSICIONAL para evitar erro de keyword)
    def create_ep(o, a, r, t):
        r = r.reshape(-1, 1) if r.ndim == 1 else r
        is_term = bool(t[-1]) if len(t) > 0 else False
        
        # CORREÇÃO: Argumentos posicionais na ordem (obs, act, rew, terminated)
        return Episode(o, a, r, is_term)

    train_episodes = [create_ep(train_obs, train_act, train_rew, train_term)]
    test_episodes = [create_ep(test_obs, test_act, test_rew, test_term)]
    
else:
    train_episodes, test_episodes = train_test_split(dataset.episodes, test_size=0.2, random_state=42)

print(f"  Episódios de Treino: {len(train_episodes)}")
print(f"  Episódios de Teste:  {len(test_episodes)}")


[3/5] Dividindo dados em Treino e Teste (Hold-out)...

⚠️ Aviso: Encontrados apenas 1 episódios.

   -> Dividindo o episódio gigante em sub-episódios menores para validação...

  Episódios de Treino: 1

  Episódios de Teste:  1

In [4]:
# ============================================================================
# CÉLULA 4: Configuração do Agente CQL (Hiperparâmetros da Metodologia)
# ============================================================================
print("\n[4/5] Configurando Agente CQL (Venda Única)...")

# Configuração alinhada com a metodologia LOCAC:
# - alpha=5.0 (Alto conservadorismo para evitar preços alucinados)
# - n_quantiles=64 (Para análise de risco VaR/CVaR)
# - gamma=0.98 (Horizonte longo)

cql_config = CQLConfig(
    batch_size=256,
    gamma=0.98,
    # Scalers já foram aplicados no Generator, então passamos None aqui
    # para o d3rlpy não tentar escalar de novo (o que estragaria os dados)
    observation_scaler=None,
    action_scaler=None,
    reward_scaler=None,
    alpha_learning_rate=1e-4,
    actor_learning_rate=1e-4,
    critic_learning_rate=3e-4,
    conservative_weight=5.0, # Peso conservador alto (Segurança)
    q_func_factory=QRQFunctionFactory(n_quantiles=64) # Q-Function Distribucional
)

# Cria o agente (usa GPU se disponível, senão CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
cql = cql_config.create(device=device)

print(f"✓ Agente CQL configurado no dispositivo: {device}")


[4/5] Configurando Agente CQL (Venda Única)...

✓ Agente CQL configurado no dispositivo: cpu

In [5]:
# ============================================================================
# CÉLULA 5: Treinamento com Governança (Manual Loop) - CORRIGIDO (Shape Fix)
# ============================================================================
import os
from d3rlpy.metrics import AverageValueEstimationEvaluator
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer, Episode

print("\n[5/5] Iniciando Treinamento Offline com Governança (Loop Manual)...")

# --- CORREÇÃO DE DADOS (FIX SHAPE) ---
# ============================================================================
# FUNÇÃO DE CORREÇÃO DE DADOS (Versão Final - Corrigida)
# ============================================================================
# ============================================================================
# FUNÇÃO DE CORREÇÃO DE DADOS (Versão Final - Robusta para Arrays)
# ============================================================================


def fix_episodes_shape(episodes):
    fixed_episodes = []
    for ep in episodes:
        # 1. Corrigir Formato das Recompensas (1D -> 2D)
        rewards = ep.rewards
        if rewards.ndim == 1:
            rewards = rewards.reshape(-1, 1)
            
        # 2. Obter flag terminal (Booleano)
        # O erro anterior mostrou que 'terminal' pode ser um array.
        # Precisamos lidar com isso.
        
        # Tenta pegar o atributo 'terminal' ou 'terminated'
        raw_terminal = getattr(ep, 'terminal', getattr(ep, 'terminated', True))
        
        # Se for um array numpy ou lista, pegamos o último elemento (que importa)
        if isinstance(raw_terminal, (np.ndarray, list)):
            # Se for array, assume que o último passo define se o episódio acabou
            # Usamos .any() se quisermos saber se houve QUALQUER terminal,
            # mas geralmente queremos saber se o episódio ACABOU no final.
            # Para segurança, vamos converter para array numpy e checar o último valor.
            term_arr = np.array(raw_terminal)
            is_terminal = bool(term_arr[-1]) if term_arr.size > 0 else True
        else:
            # Se já for escalar (bool/int/float), converte direto
            is_terminal = bool(raw_terminal)
        
        # 3. Criar novo Episódio (POSICIONAL)
        # A ordem: (observations, actions, rewards, terminated)
        fixed_episodes.append(Episode(
            ep.observations,  # 1. Observações
            ep.actions,       # 2. Ações
            rewards,          # 3. Recompensas (Corrigidas para 2D)
            is_terminal       # 4. Terminated (BOOLEANO LIMPO)
        ))
            
    return fixed_episodes

print("   -> Corrigindo formato dos dados (Rewards 2D + Robust Terminal Check)...")
train_episodes = fix_episodes_shape(train_episodes)
test_episodes = fix_episodes_shape(test_episodes)
print("   -> Dados corrigidos com sucesso.")

# Configurações de Governança
N_STEPS = 50000           
STEPS_PER_EPOCH = 1000    
PATIENCE = 20             
BEST_MODEL_PATH = "modelo_rl_final.pt"

# 1. Criar o Avaliador (Evaluator)
avg_q_evaluator = AverageValueEstimationEvaluator(test_episodes)

# 2. Criar o dicionário de avaliadores
evaluators = {
    'average_q': avg_q_evaluator
}

# 3. Criar o Buffer de Treino (com dados corrigidos)
train_buffer = ReplayBuffer(
    FIFOBuffer(limit=dataset.transition_count),
    episodes=train_episodes
)

# Estado inicial
best_score = -float('inf')
patience_counter = 0

print(f"  -> Treinando por até {N_STEPS} passos")
print(f"  -> Avaliação a cada {STEPS_PER_EPOCH} passos")
print(f"  -> Early Stopping: {PATIENCE} épocas sem melhora")

# Loop de Treino
for epoch, metrics in cql.fitter(
    train_buffer,
    n_steps=N_STEPS,
    n_steps_per_epoch=STEPS_PER_EPOCH,
    evaluators=evaluators,          
    experiment_name="cql_venda_unica_run",
    with_timestamp=False,
    show_progress=True
):
    # 1. Obter a métrica de interesse
    current_score = metrics.get('average_q')
    
    # 2. Lógica de Model Checkpoint
    if current_score > best_score:
        best_score = current_score
        patience_counter = 0
        cql.save_model(BEST_MODEL_PATH)
        print(f"    [Epoch {epoch}] Novo recorde! Score: {current_score:.4f} -> Modelo salvo.")
    else:
        patience_counter += 1
        print(f"    [Epoch {epoch}] Sem melhora. Score: {current_score:.4f} (Paciência: {patience_counter}/{PATIENCE})")
        
    # 3. Lógica de Early Stopping
    if patience_counter >= PATIENCE:
        print(f"\n🛑 EARLY STOPPING ACIONADO na época {epoch}.")
        print("   O modelo parou de aprender. Encerrando treino para evitar overfitting.")
        break

# Pós-Treino
if os.path.exists(BEST_MODEL_PATH):
    cql.load_model(BEST_MODEL_PATH)
    print(f"\n✓ Treino finalizado. Melhor modelo carregado de '{BEST_MODEL_PATH}'.")
else:
    print("\n⚠️ Aviso: Nenhum modelo foi salvo.")


[5/5] Iniciando Treinamento Offline com Governança (Loop Manual)...

   -> Corrigindo formato dos dados (Rewards 2D + Robust Terminal Check)...

   -> Dados corrigidos com sucesso.

2025-11-27 13:57.59

 [

...

Model parameters are saved to d3rlpy_logs\cql_venda_unica_run\model_21000.d3

    [Epoch 21] Sem melhora. Score: -21558.3356 (Paciência: 20/20)


🛑 EARLY STOPPING ACIONADO na época 21.

   O modelo parou de aprender. Encerrando treino para evitar overfitting.


✓ Treino finalizado. Melhor modelo carregado de 'modelo_rl_final.pt'.

In [6]:
# ============================================================================
# CÉLULA 6: Avaliação Final e Métricas de Negócio - CORREÇÃO FINAL (Dimensão)
# ============================================================================
import os
import numpy as np
from d3rlpy.metrics import AverageValueEstimationEvaluator

print("\n" + "="*40)
print("AVALIAÇÃO FINAL E MÉTRICAS DE NEGÓCIO")
print("="*40)

# 1. Carregar o melhor modelo salvo
if os.path.exists("modelo_rl_final.pt"):
    cql.load_model("modelo_rl_final.pt")
    print("✓ Melhor modelo ('modelo_rl_final.pt') carregado com sucesso.")
else:
    print("⚠️ Aviso: 'modelo_rl_final.pt' não encontrado. Usando modelo atual da memória.")

# 2. Calcular o Valor da Política
print("Calculando valor da política...")
estimator = AverageValueEstimationEvaluator(test_episodes)
valor_politica_norm = estimator(cql, dataset=dataset)

# Converter para Dólares
valor_politica_real = scaler_recompensa.inverse_transform([[valor_politica_norm]])[0][0]

print(f"\n--- Performance Global (Conjunto de Teste) ---")
print(f"  Valor da Política (Normalizado): {valor_politica_norm:.4f}")
print(f"  => LUCRO MÉDIO ESTIMADO POR VENDA: ${valor_politica_real:.2f}")

# 3. Inferência de Exemplo
print(f"\n--- Exemplo de Inferência (Um Caso do Teste) ---")

sample_episode = test_episodes[0]
sample_state = sample_episode.observations[0]

# Garante input 2D para o modelo
state_batch = sample_state.reshape(1, -1)

# A. Prever Ação (Retorna array 1D com a ação)
acao_norm = cql.predict(state_batch)[0]

# B. Converter para Real (CORREÇÃO AQUI: Reshape direto, sem colchetes extras)
preco_real = scaler_acao.inverse_transform(acao_norm.reshape(1, -1))[0][0]

# C. Calcular Lucro Esperado
if hasattr(cql, "predict_value"):
    # Prepara ação em batch 2D
    action_batch = acao_norm.reshape(1, -1)
    lucro_esperado_norm = cql.predict_value(state_batch, action_batch)[0]
    # Correção também aqui para garantir 2D
    lucro_esperado_real = scaler_recompensa.inverse_transform(lucro_esperado_norm.reshape(1, -1))[0][0]
else:
    lucro_esperado_real = 0.0 

print(f"  Estado (Shape): {sample_state.shape}")
print(f"  Ação do Modelo (Normalizada): {acao_norm}")
print(f"  ------------------------------------------------")
print(f"  ✅ PREÇO RECOMENDADO: ${preco_real:.2f}")
print(f"  💰 LUCRO ESPERADO:    ${lucro_esperado_real:.2f}")

if preco_real < 0:
    print("\n❌ ALERTA: O preço recomendado é negativo. Verifique o 'Generator_NEW.py'.")
else:
    print("\n✅ SUCESSO: O sistema está operando corretamente!")

AVALIAÇÃO FINAL E MÉTRICAS DE NEGÓCIO

✓ Melhor modelo ('modelo_rl_final.pt') carregado com sucesso.

Calculando valor da política...


--- Performance Global (Conjunto de Teste) ---

  Valor da Política (Normalizado): -0.1091

...

  Ação do Modelo (Normalizada): [-0.8670802]

  ------------------------------------------------

  ✅ PREÇO RECOMENDADO: $-7.57

  💰 LUCRO ESPERADO:    $2874.63


❌ ALERTA: O preço recomendado é negativo. Verifique o 'Generator_NEW.py'.